In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .master("local[4]") \
        .appName("homework") \
        .getOrCreate()

25/03/03 06:47:26 WARN Utils: Your hostname, furix-Z690-UD-DDR4 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface enp9s0)
25/03/03 06:47:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 06:47:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-03 07:47:55--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2156:c200:b:20a5:b140:21, 2600:9000:2156:0:b:20a5:b140:21, 2600:9000:2156:f800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2156:c200:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  37.7MB/s    in 1.6s    

2025-03-03 07:47:58 (37.7 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [5]:
df= spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [7]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df.repartition(4).write.parquet('./data/')

In [9]:
!ls -lh data/

total 90M
-rw-r--r-- 1 furix furix 23M Mar  3 07:50 part-00000-2160ef99-5386-483a-90f2-7615e9a460ce-c000.snappy.parquet
-rw-r--r-- 1 furix furix 23M Mar  3 07:50 part-00001-2160ef99-5386-483a-90f2-7615e9a460ce-c000.snappy.parquet
-rw-r--r-- 1 furix furix 23M Mar  3 07:50 part-00002-2160ef99-5386-483a-90f2-7615e9a460ce-c000.snappy.parquet
-rw-r--r-- 1 furix furix 23M Mar  3 07:50 part-00003-2160ef99-5386-483a-90f2-7615e9a460ce-c000.snappy.parquet
-rw-r--r-- 1 furix furix   0 Mar  3 07:50 _SUCCESS


In [10]:
df = spark.read.parquet('data/*')

In [11]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [23]:
from pyspark.sql.functions import to_date, col, lit

df.select(to_date(col("tpep_pickup_datetime")).alias('tpep_pickup_datetime')) \
    .groupBy('tpep_pickup_datetime').count() \
    .filter(col("tpep_pickup_datetime") == to_date(lit('2024-10-15'))).show()


+--------------------+------+
|tpep_pickup_datetime| count|
+--------------------+------+
|          2024-10-15|128893|
+--------------------+------+



In [24]:
df.createOrReplaceTempView("df")

spark.sql("""
    SELECT TO_DATE(tpep_pickup_datetime) AS tpep_pickup_datetime, COUNT(*) AS count
    FROM df
    GROUP BY TO_DATE(tpep_pickup_datetime)
    HAVING tpep_pickup_datetime = '2024-10-15'
""").show()


+--------------------+------+
|tpep_pickup_datetime| count|
+--------------------+------+
|          2024-10-15|128893|
+--------------------+------+



In [30]:
df.createOrReplaceTempView("df")
spark.sql(""" 
          SELECT MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) AS max_duration_hours
          FROM df;
          """).show()

+------------------+
|max_duration_hours|
+------------------+
|162.61777777777777|
+------------------+



In [31]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-03 08:20:48--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2156:8800:b:20a5:b140:21, 2600:9000:2156:7c00:b:20a5:b140:21, 2600:9000:2156:c600:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2156:8800:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.008s  

2025-03-03 08:20:49 (1.42 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [35]:
df_zones = spark.read.csv('taxi_zone_lookup.csv',header=True)

In [36]:
df_zones.show(1)

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
+----------+-------+--------------+------------+
only showing top 1 row



In [37]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [40]:
from pyspark.sql.types import IntegerType
df_zones = df_zones.withColumn('LocationID',col('LocationID').cast(IntegerType()))

In [41]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [45]:
df.createOrReplaceTempView("trips")
df_zones.createOrReplaceTempView("zones")
spark.sql("""
          SELECT zones.Zone as pickup_zone, count(*) as trip_count
          FROM trips
          left join zones on trips.PULocationID = zones.LocationID
          GROUP BY pickup_zone
          ORDER BY trip_count ASC
          """).show()

+--------------------+----------+
|         pickup_zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
|       Rikers Island|         2|
|       Arden Heights|         2|
|         Jamaica Bay|         3|
| Green-Wood Cemetery|         3|
|Charleston/Totten...|         4|
|   Rossville/Woodrow|         4|
|Eltingville/Annad...|         4|
|       West Brighton|         4|
|       Port Richmond|         4|
|         Great Kills|         6|
|        Crotona Park|         6|
|     Mariners Harbor|         7|
|Heartland Village...|         7|
|Saint George/New ...|         9|
|             Oakwood|         9|
|       Broad Channel|        10|
|New Dorp/Midland ...|        10|
|         Westerleigh|        12|
|     Pelham Bay Park|        12|
+--------------------+----------+
only showing top 20 rows

